# Notebook for analyzing the results

In this notebook we analyze the results from the experiments we ran. 

In [4]:
import json, glob
import pandas as pd

Store the records as a dataframe

In [ ]:
records = []
for f in glob.glob("results/**/*.json", recursive=True):
    with open(f) as fp:
        r = json.load(fp)
        record = {
            "algorithm": r["algorithm"],
            "network": r["network"].split("/")[-1].split(".")[0],
            "num_samples": r["num_samples"],
            "score": r["score"],
            "runtime": r["seconds_elapsed"],
            "k": r["params"].get("k"),
            "l": r["params"].get("l"),
            "num_vars": r["num_variables"],
        }
        records.append(record)

df = pd.DataFrame(records)

In [6]:
for record in df.iterrows():
    print(record)